In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_json('train.json', lines=True)
test = pd.read_json('test.json', lines=True)
public = test.query('seq_length == 107').copy()
private = test.query('seq_length == 130').copy()
print(train.shape)
print(public.shape)
print(private.shape)

(2400, 19)
(629, 7)
(3005, 7)


In [4]:
def process_seq(data, seq_length):
    # processes the 3 sequences and the target
    seq = data.loc[:, 'sequence'].str.slice(0, seq_length).values
    struct = data.loc[:, 'structure'].str.slice(0, seq_length).values
    loop = data.loc[:, 'predicted_loop_type'].str.slice(0, seq_length).values
    
    n = data.shape[0]

    new_seq = []
    new_struct = []
    new_loop = []

    for i in range(n):
        # sequence processing
        seq_dic = {'A': 0, 'C': 1, 'G': 2, 'U': 3} 
        n_seq = [seq_dic[l] for l in seq[i]]
        new_seq.append(n_seq)

        # structure processing
        struct_dic = {'(': 4, '.': 5, ')': 6}
        n_struct = [struct_dic[l] for l in struct[i]]
        new_struct.append(n_struct)

        # predicted_loop_type processing
        loop_dic = {'S':7, 'M':8, 'I':9, 'B':10, 'H':11, 'E':12, 'X':13}
        n_loop = [loop_dic[l] for l in loop[i]]
        new_loop.append(n_loop)
        
    new_seq = np.array(new_seq)
    new_struct = np.array(new_struct)
    new_loop = np.array(new_loop)
        
    new_seq = new_seq.reshape(new_seq.shape[0], new_seq.shape[1], 1) 
    new_struct = new_struct.reshape(new_struct.shape[0], new_struct.shape[1], 1)
    new_loop = new_loop.reshape(new_loop.shape[0], new_loop.shape[1], 1)
    
    return np.concatenate([new_seq, new_struct, new_loop], axis=2)

def process_target(data, seq_scored):
    n = data.shape[0]
    y_train = data.loc[:, ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']]
    new_target = np.zeros((n, 5, seq_scored))
    
    for i in range(n):
        new_target[i, :, :] = np.concatenate(y_train.values[i]).reshape(5, seq_scored)
    # swap the 2 last axes to match sequence shape
    new_target = np.swapaxes(new_target, 1, 2)
    return new_target

In [5]:
features = process_seq(train, seq_length=107)
features_public = process_seq(public, seq_length=107)
features_private = process_seq(private, seq_length=130)

target = process_target(train, seq_scored=68)

print(features.shape)
print(features_public.shape)
print(features_private.shape)
print(target.shape)
# features of the first 4 bases of the first sequence
print(features[0, :4, :])

(2400, 107, 3)
(629, 107, 3)
(3005, 130, 3)
(2400, 68, 5)
[[ 2  5 12]
 [ 2  5 12]
 [ 0  5 12]
 [ 0  5 12]]


In [7]:
def get_bpps(ids):
    # retrieve bpps matrices
    bpps = []
    for id in ids:
        bpps.append(np.load(f"bpps/{id}.npy"),)
    return np.array(bpps)


X_bpps = get_bpps(train.id.values)
X_bpps_public = get_bpps(public.id.values)
X_bpps_private = get_bpps(private.id.values)
print(X_bpps.shape)
print(X_bpps_public.shape)
print(X_bpps_private.shape)

(2400, 107, 107)
(629, 107, 107)
(3005, 130, 130)


In [8]:
def min_max_norm(m):
    # min max scaling for non negatives values
    maxi = np.amax(m) + 1e-8  # avoid dividing by 0
    return m / maxi


def get_nonzeros(X_bpps):
    # return non 0 proportion of each row of the bpps files
    X_bpps_nonzeros_prop = []
    n = X_bpps.shape[0]
    seq_length = X_bpps.shape[1]
    for k in range(n):
        X_bpps_nonzeros_prop.append(1 - ((X_bpps[k] == 0).sum(axis=1) / seq_length))
    X_bpps_nonzeros_prop = np.array(X_bpps_nonzeros_prop)
    return X_bpps_nonzeros_prop


def get_dist(structures, seq_length):
    # return distance matrix according to the distance between paired bases
    dist = np.zeros((structures.shape[0], seq_length, seq_length))
    idx = []
    for row, structure in enumerate(structures):
        for i, token in enumerate(structure):
            if token == "(":
                idx.append(i)
            elif token == ")":
                j = idx.pop()  # index of the corresponding '(' 
                dist[row, i, j] = i-j
                dist[row, j, i] = i-j
    return dist


def get_features(data, seq_length):
    # return the stats features from bpps files
    X_bpps = get_bpps(data.id.values)  # retrieve bpps matrices
    X_dist = get_dist(data['structure'].str.slice(0, seq_length), seq_length)
    X_bpps_nonzeros_prop = get_nonzeros(X_bpps)

    train_bpps_stats = [min_max_norm(X_bpps.mean(axis=2)),  # mean of non zeros values for each base
                        min_max_norm(X_bpps.max(axis=2)),  # max value for each base
                        min_max_norm(X_bpps_nonzeros_prop),  # proportion of non zeros for each base
                        min_max_norm(X_dist.sum(axis=2))]  # distance of the base to its partner 
    X_stats = np.concatenate([stats[:,:,None] for stats in train_bpps_stats], axis=2)
    return X_stats, X_bpps, X_dist

# bpps stats features
X_stats, X_bpps, X_dist = get_features(train, seq_length=107)
X_stats_public, X_bpps_public, X_dist_public = get_features(public, seq_length=107)
X_stats_private, X_bpps_private, X_dist_private = get_features(private, seq_length=130)

# concatenate sequences features and bpps stats features
X = np.concatenate([features, X_stats], axis=2)
np.save('X.npy', X)
X_public = np.concatenate([features_public, X_stats_public], axis=2)
np.save('X_public.npy', X_public)
X_private = np.concatenate([features_private, X_stats_private], axis=2)
np.save('X_private.npy', X_private)
y = target
np.save('y.npy', y)

In [9]:
# split data
# lets select validation indexes where SN_filter = 1. 
# Indeed private test sequences are based on that type of filter
np.random.seed(42)
idx_filter = np.where(train.SN_filter == 1)[0]
choice = np.random.choice(idx_filter, 256, replace=False)
idx_valid = np.ones(X.shape[0], dtype=np.bool)
idx_valid[choice] = False

X_train = X[idx_valid, :, :]
X_valid = X[~idx_valid, :, :]
y_train = y[idx_valid]
y_valid = y[~idx_valid]


print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)
print(X_public.shape)
print(X_private.shape)
print(X_train[0, :4, 3:])  # stats for the first 4 bases of the first sequence

(2144, 107, 7)
(256, 107, 7)
(2144, 68, 5)
(256, 68, 5)
(629, 107, 7)
(3005, 130, 7)
[[0.19854377 0.02178589 0.31818181 0.        ]
 [0.18371357 0.03865303 0.22727272 0.        ]
 [0.06000285 0.02759063 0.05681818 0.        ]
 [0.01312231 0.00947074 0.04545454 0.        ]]


## Random Forest

In [10]:
def make_random_forest(X_train, y_train, X_valid, y_valid):
    X_trn = X_train[:, :68, :].reshape(X_train.shape[0], 68*X_train.shape[2])
    X_val = X_valid[:, :68, :].reshape(X_valid.shape[0], 68*X_valid.shape[2])
    y_trn = y_train.reshape(y_train.shape[0], y_train.shape[1]*y_train.shape[2])

    reg = RandomForestRegressor(n_estimators=2000, 
                                max_features='sqrt',
                                criterion='mse',
                                min_samples_leaf=2, 
                                max_depth=40,
                                n_jobs=-1,  # activate all the cores of the cpu
                                random_state=42,
                                verbose=1)  
    reg.fit(X_trn, y_trn)
    y_pred = reg.predict(X_val)
    return y_pred

In [11]:
y_pred = make_random_forest(X_train, y_train, X_valid, y_valid)
y_val = y_valid.reshape(y_valid.shape[0], y_valid.shape[1]*y_valid.shape[2])
y_pred[y_pred < 0.0] = 0.0
y_pred[y_pred > 6.0] = 6.0
rmse = np.sqrt(mean_squared_error(y_val, y_pred))  # best 0.3404 - 2000 trees
rmse  

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  1.8min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Do

0.3336529170407562